In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [2]:
#=========
# Setup
#=========
orders = pd.DataFrame({
    "order_id": [101 , 102 , 103 , 104 , 105] ,
    "customer_id": [1 , 2 , 3 , 2 , 99] ,
    "amount": [120 , 250 , 80 , 60 , 90] ,
})
orders

customers = pd.DataFrame({
    "customer_id": [1 , 2 , 3] ,
    "region": ["East" , "West" , "East"] ,
    "segment": ["Retail" , "B2B" , "Retail"] ,
})
customers

returns = pd.DataFrame({
    "order_id": [102 , 104 , 104] ,
    "reason": ["damaged" , "late" , "wrong_item"] ,
})
returns

,order_id,customer_id,amount
0,101,1,120
1,102,2,250
2,103,3,80
3,104,2,60
4,105,99,90


,customer_id,region,segment
0,1,East,Retail
1,2,West,B2B
2,3,East,Retail


,order_id,reason
0,102,damaged
1,104,late
2,104,wrong_item


In [3]:
#====================================
# Case 1) merge(): SQL-style join
#====================================
case1 = orders.merge(customers , on = "customer_id" , how = "inner")
case1

,order_id,customer_id,amount,region,segment
0,101,1,120,East,Retail
1,102,2,250,West,B2B
2,103,3,80,East,Retail
3,104,2,60,West,B2B


In [4]:
#============================================================
# Case 2) how= (left vs outer) controls row preservation
#============================================================
case2_left = orders.merge(customers , on = "customer_id" , how = "left")
case2_left

case2_outer = orders.merge(customers , on = "customer_id" , how = "outer")
case2_outer

,order_id,customer_id,amount,region,segment
0,101,1,120,East,Retail
1,102,2,250,West,B2B
2,103,3,80,East,Retail
3,104,2,60,West,B2B
4,105,99,90,NaN,NaN


,order_id,customer_id,amount,region,segment
0,101,1,120,East,Retail
1,102,2,250,West,B2B
2,104,2,60,West,B2B
3,103,3,80,East,Retail
4,105,99,90,NaN,NaN


In [6]:
#============================================================
# Case 3) indicator=True (which table contains the record)
#============================================================
case3 = orders.merge(customers , on = "customer_id" , how = "left" , indicator = True)
case3

,order_id,customer_id,amount,region,segment,_merge
0,101,1,120,East,Retail,both
1,102,2,250,West,B2B,both
2,103,3,80,East,Retail,both
3,104,2,60,West,B2B,both
4,105,99,90,NaN,NaN,left_only


In [8]:
#====================================================================
# Case 4) validate= to catch unexpected duplicates / row explosion
# (This will raise if relationship assumption is violated)
#====================================================================
try:
    # orders: unique order_id
    # returns: order_id has duplicates (104 appears twice)
    # so one_to_one should FAIL (as intended)
    case4 = orders.merge(returns , on = "order_id" , how = "left" , validate = "one_to_one")
except Exception as e:
    print("\nValidate='one_to_one' exception raised (expected):")
    print(type(e).__name__ , ":" , e)

# A safer choice: one_to_many (left unique, right may have duplicates)
case4_ok = orders.merge(returns , on = "order_id" , how = "left" , validate = "one_to_many")
print("\nValidate='one_to_many' works:")
print(case4_ok)


Validate='one_to_one' exception raised (expected):
MergeError : Merge keys are not unique in right dataset; not a one-to-one merge

Validate='one_to_many' works:
   order_id  customer_id  amount      reason
0       101            1     120         NaN
1       102            2     250     damaged
2       103            3      80         NaN
3       104            2      60        late
4       104            2      60  wrong_item
5       105           99      90         NaN


In [9]:
#===========================================
# Case 5) join(): index-based lookup join
#===========================================
cust_lu = customers.set_index("customer_id")[["region" , "segment"]]
case5 = orders.join(cust_lu , on = "customer_id")
case5

,order_id,customer_id,amount,region,segment
0,101,1,120,East,Retail
1,102,2,250,West,B2B
2,103,3,80,East,Retail
3,104,2,60,West,B2B
4,105,99,90,NaN,NaN


In [10]:
#============================================================
# Case 6) merge_asof(): time-aware join (nearest/last-known)
# IMPORTANT: both frames must be sorted by the key
#============================================================
events = pd.DataFrame({
    "ts": pd.to_datetime([
        "2025-12-01 09:05" , "2025-12-01 09:12" , "2025-12-01 09:30" , "2025-12-01 10:02" ,
    ]) ,
    "event": ["open" , "view" , "add_to_cart" , "checkout"] ,
}).sort_values("ts")
events

signals = pd.DataFrame({
    "ts": pd.to_datetime([
        "2025-12-01 09:00" , "2025-12-01 09:20" , "2025-12-01 10:00" ,
    ]) ,
    "site_latency_ms": [180 , 240 , 190] ,
}).sort_values("ts")
signals

case6 = pd.merge_asof(events , signals , on = "ts" , direction = "backward")
case6

,ts,event
0,2025-12-01 09:05:00,open
1,2025-12-01 09:12:00,view
2,2025-12-01 09:30:00,add_to_cart
3,2025-12-01 10:02:00,checkout


,ts,site_latency_ms
0,2025-12-01 09:00:00,180
1,2025-12-01 09:20:00,240
2,2025-12-01 10:00:00,190


,ts,event,site_latency_ms
0,2025-12-01 09:05:00,open,180
1,2025-12-01 09:12:00,view,180
2,2025-12-01 09:30:00,add_to_cart,240
3,2025-12-01 10:02:00,checkout,190
